In [1]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os 

from sklearn.model_selection import train_test_split
from sklearn.metrics import completeness_score, homogeneity_score

In [ ]:
#2A
def cosSim(vecA, vecB):
    #D_norm = array([linalg.norm(data[i]) for i in range(len(data))])
    #x_norm = linalg.norm(inX)
    #cosines = dot(data,inX)/(D_norm * x_norm)
    #distances = 1 - cosines
    
    norm_A = linalg.norm(vecA)
    norm_B = linalg.norm(vecB)
    cosine = dot(vecA, vecB) / (norm_A * norm_B)
    distance = 1 - cosine
    return distance

In [ ]:
#2B
matrix = pd.DataFrame(np.genfromtxt(r"C:\Users\Owner\Documents\DSC 478\matrix.txt", delimiter=',', dtype=int).T)
classes = pd.DataFrame(np.genfromtxt(r"C:\Users\Owner\Documents\DSC 478\classes.txt", delimiter=' ', dtype=int, skip_header=1))
terms = np.array(pd.DataFrame(np.genfromtxt(r"C:\Users\Owner\Documents\DSC 478\terms.txt", delimiter='\t', dtype=str)))

In [4]:
matrix.shape

(2500, 9328)

In [5]:
terms.shape

(9328, 1)

In [6]:
classes = classes.drop(classes.columns[0], axis=1)
classes.head()

,1
0,0
1,1
2,1
3,1
4,2


In [5]:
train, test, classes_train, classes_test = train_test_split(matrix, classes, test_size=0.2, random_state=33)
print(train.shape, test.shape, classes_train.shape, classes_test.shape)

(2000, 9328) (500, 9328) (2000, 2) (500, 2)


In [24]:
#2.c
dt = train
td = train.T

dt_test = test
td_test = test.T

df = np.array([(td!=0).sum(1)]).T

matrix_doc_number = dt.shape[0]
matrix_term_number = dt.shape[1]
term_freq = train.sum(axis = 1)

newMatrix = np.ones(np.shape(td), dtype = float) * matrix_doc_number

idf = np.log2(np.divide(newMatrix, df))

td_tfidf_train = np.array(td * idf)
td_tfidf_test = np.array(td_test * idf[:, :500])
dt_tfidf_train = td_tfidf_train.T
dt_tfidf_test = td_tfidf_test.T

dt_tfidf_train[np.isnan(dt_tfidf_train)] = 0
dt_tfidf_test[np.isnan(dt_tfidf_test)] = 0
dt_tfidf_train[dt_tfidf_train == inf] = 0
dt_tfidf_train[dt_tfidf_train == -inf] = 0
dt_tfidf_test[dt_tfidf_test == inf] = 0
dt_tfidf_test[dt_tfidf_test == -inf] = 0

<ipython-input-24-6d75cd11c952>:16: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log2(np.divide(newMatrix, df))


In [18]:
from numpy import *

def distEuc(va, vb):
    return sqrt(sum(power(va - vb, 2)))

def randomCent(ds, k):
    n = shape(ds)[1]
    cent = zeros((k,n), dtype = float)
    for j in range(n):
        j_min = min(ds[:, j])
        j_range = float(max(ds[:, j]) - j_min)
        cent[:, j] = j_min + j_range * random.rand(k)    
    return cent

def kMeans(ds, k, distMeas = cosSim, createCent = randomCent):
    m = shape(ds)[0]
    cent = createCent(ds, k)
    c_assment = zeros((m, 2))
    c_changed = True
    while c_changed:
        c_changed = False
        for i in range(m):
            min_dist = inf 
            min_index = -1
            for j in range(k):
                ji_dist = distMeas(cent[j, :],ds[i, :])
                if ji_dist < min_dist:
                    min_dist = ji_dist 
                    min_index = j
            if c_assment[i,0] != min_index: 
                c_changed = True
            c_assment[i, :] = min_index, min_dist ** 2
        for c in range(k):
            pts = ds[nonzero(c_assment[:, 0] == c)[0]]
            if(len(pts)!=0):
                cent[c,:] = mean(pts, axis=0)
    return cent, c_assment

In [ ]:
def top_terms(data, K, numterms):
    cent, c_assment = kMeans(mat(data), K, distMeas = cosSim)
    for i in range(K):
        print('Cluster', i+1, ': ')
        c_matrix = data[c_assment[:, 0] == i]
        print('Number of items in Cluster ',i+1 ,': ', c_matrix.shape[0])
        c_freq = np.array([(c_matrix.T != 0).sum(1)]).T
        
        mc = c_freq / float(c_matrix.shape[0])
        mc = map(list, mc)
        mc = [m[0] for m in mc]
        c_freq = [m[0] for m in c_freq]
        n = sorted(zip(terms, c_freq, mc),key = lambda x:x[2], reverse = True)
        for m in n[:numterms]:
            print('Term: ',m[0])
            print('Frequency: ',m[1])
            print('Percent of docs contain term ', m[0], ': ', m[2] * 100,'\n')
        print('\n')
    return cent, c_assment

cent, c_assment = top_terms(dt_tfidf_train, 5, 10)

In [ ]:
#2D
from sklearn.metrics import completeness_score, homogeneity_score

cs = completeness_score(np.array(classes_train.T)[0],c_assment[:,0])
hs = homogeneity_score(np.array(classes_train.T)[0],c_assment[:,0])

print('Completeness score: %0.8f' % cs)
print('Homogeneity of Clusters: %0.8f' % hs)

In [ ]:
#2E
results = []
for d in dt_tfidf_test:  
    dic = {}
    sim = []
    c = 0
    for centroid in cent:
        c += 1
        cmatrix_sim = 1 - cosineSim(d, centroid)
        sim.append(cmatrix_sim) 
        dic['Cosine Similarity' + str(c)] = cmatrix_sim
    dic['Predicted Cluster'] = sim.index(max(sim)) + 1
    results.append(dic)
test_cluster = pd.DataFrame(results)

In [ ]:
test_cluster.head(5)